In [ ]:
!pip install --upgrade --no-cache-dir -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple eazyml-genai
!pip install gdown python-dotenv

## Import libraries

In [ ]:
import os
from eazyml_genai.components import PDFLoader
from eazyml_genai.components import PineconeDB
from eazyml_genai.components import GoogleGM
from eazyml_genai.components import(
    OpenAIEmbeddingModel,
    GoogleEmbeddingModel,
    HuggingfaceEmbeddingModel,
    HuggingfaceEmbeddingProcessor
)

from dotenv import load_dotenv
load_dotenv()

## Load data

### Read PDF documents
process pdf documents with unstructured data into semi-structured data or in json format

In [ ]:
pdf_loader = PDFLoader(max_chunk_words=1000)
documents = pdf_loader.load(file_path=r'data/yolo.pdf')

## Set API Key

In [ ]:
# set your api key below
os.environ['GEMINI_API_KEY'] = os.getenv('GEMINI_API_KEY')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = os.getenv('PINECONE_API_KEY')

## Index data

### Index Document
Pinecone vector database could be initialized in 2 ways:

1. ```python
    pine_db = PineconeDB(api_key=os.getenv("PINECONE_API_KEY"))
    ```
2. ```python
    pine_db = PineconeDB(api_key="pclocal", host="http://localhost:5080")
    ```

In [ ]:
pine_db = PineconeDB(api_key="pclocal", host="http://localhost:5080")

In [ ]:
collection_name = 'yolo'
indexed_documents = pine_db.index_documents(
                                collection_name=collection_name,
                                documents=documents,
                                text_embedding_model=HuggingfaceEmbeddingModel.ALL_MINILM_L6_V2,
                        )

## Generate Answer using Generative Model

### Retrieve documents based on given question

In [ ]:
question = "number of convolutional layer"
results = pine_db.retrieve_documents(
                                question=question
                                )

In [ ]:
payloads = [i['metadata'] for i in results]

### Generate answer for question and retrieved documents

In [ ]:
google_gm = GoogleGM(model="gemini-2.0-flash",
                     api_key=os.getenv('GEMINI_API_KEY'))
response, input_tokens, output_tokens = google_gm.predict(question=question,
                            payloads=payloads,
                            show_token_details=True
                            )

In [ ]:
parsed_response = google_gm.parse(response=response)

In [ ]:
print(parsed_response)

In [ ]:
input_tokens